In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## Reading in Dataset

### Fixed Cost of each WTP

In [2]:
fixed_cost = pd.read_excel("../dataset2/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Fixed Cost
WTP,
Ampang Intake,11638
Batang Kali,11011
Bernam River Head,11602
WTP1,4899
WTP2,11773


### Capacity of each WTP

In [3]:
capacity = pd.read_excel("../dataset2/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,2.641160e+07
Batang Kali,2.858907e+07
Bernam River Head,2.993616e+07
WTP1,2.728402e+07
WTP2,2.898516e+07


In [4]:
capacity.loc["WTP1", "Capacity"]

27284019.78365

### Distribution Loss

In [5]:
distribution_loss = pd.read_excel("../dataset2/distribution_loss.xlsx", index_col=0)
distribution_loss.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.31838,0.34819,0.23241,0.26049,0.34462,0.28837,0.30351,0.28379,0.33333,0.30272,...,0.29068,0.28830,0.29829,0.35956,0.26697,0.30082,0.27045,0.28345,0.27237,0.33081
DMZ002,0.30544,0.32145,0.29041,0.26520,0.30354,0.25078,0.31171,0.26908,0.24993,0.34083,...,0.31445,0.24484,0.35810,0.34966,0.31344,0.25027,0.29574,0.27309,0.32620,0.30501
DMZ003,0.32782,0.29775,0.30906,0.33927,0.31701,0.29337,0.26027,0.31059,0.30448,0.32909,...,0.24614,0.32459,0.23055,0.31726,0.31519,0.31940,0.29601,0.28883,0.33544,0.26115
DMZ004,0.35452,0.28941,0.25286,0.30884,0.32873,0.27913,0.26008,0.33860,0.32277,0.28385,...,0.32659,0.26638,0.28412,0.34979,0.26636,0.28850,0.28361,0.29359,0.25466,0.26176
DMZ005,0.28498,0.28067,0.29266,0.32907,0.29612,0.33973,0.24868,0.34067,0.25028,0.28086,...,0.31274,0.31326,0.29827,0.27726,0.29009,0.23659,0.27612,0.35060,0.32065,0.27252


In [6]:
distribution_loss.loc["DMZ001", "WTP1"]

0.26049

### Links between WTP and DMZ

In [7]:
linkage = pd.read_excel("../dataset2/linkage2.xlsx", index_col=0)
linkage.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
DMZ002,1,1,1,1,0,1,1,1,1,1,...,1,0,1,1,1,0,1,1,0,0
DMZ003,1,1,1,1,1,1,1,1,0,1,...,1,1,0,1,1,1,1,1,0,1
DMZ004,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,1,1,1,1,0,1
DMZ005,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,0


### Variable Cost for each combination of (DMZ, WTP)

In [8]:
variable_cost = pd.read_excel("../dataset2/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,9,14,9,12,24,13,21,20,8,10,...,21,23,11,9,17,7,19,18,5,22
DMZ002,12,15,23,11,18,23,17,7,7,22,...,9,9,10,23,13,18,9,15,9,11
DMZ003,5,20,19,8,9,17,7,12,18,9,...,21,21,19,15,15,5,6,16,11,21
DMZ004,22,14,15,6,12,8,9,19,17,11,...,15,9,9,24,20,8,19,13,15,5
DMZ005,22,10,7,21,22,12,21,20,9,18,...,8,14,18,23,20,14,7,5,6,5


### Transport Cost for each combination of (DMZ, WTP)

In [57]:
# transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
# transport_cost.head()

### Demand for each DMZ

In [58]:
demand = pd.read_excel("../dataset2/demand_chisq.xlsx", index_col=0)
demand.head()

,Demand
DMZ001,160044.62961
DMZ002,150056.74766
DMZ003,141891.56807
DMZ004,194644.00762
DMZ005,126901.55527


In [59]:
demand = demand.iloc[:1100,:]

In [60]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [15]:
# demand["Demand"] = pd.to_numeric(demand["Demand"], downcast="float")
# demand.head()

## Optimisation

In [61]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [62]:
print(wtp[:5])
print("Number of WTP:", len(wtp) )

print(dmz[:5])
print("Number of DMZ:", len(dmz) )

print("Number of DMZ-WTP Pairs:", len(dmz_wtp_pairs))

['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']
Number of WTP: 45
['DMZ001', 'DMZ002', 'DMZ003', 'DMZ004', 'DMZ005']
Number of DMZ: 1100
Number of DMZ-WTP Pairs: 49500


In [63]:
demand.loc[dmz[0], "Demand"]

160044.62961

In [64]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [65]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [66]:
# fixed_cost.loc[wtp[0], "Fixed Cost"]

In [67]:
output[("DMZ001", "WTP1")]

Volume_('DMZ001',_'WTP1')

In [68]:
# Define the Objective Function

## Without transport cost
model += \
     lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])


## With transport cost
# model += \
#      lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
#      lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [69]:
curr_dmz=dmz[1]
curr_wtp=wtp[0]

In [70]:
linkage.loc[curr_dmz, curr_wtp]

1

In [71]:
1- distribution_loss.loc[curr_dmz, curr_wtp]

0.6945600000000001

In [72]:
output[(curr_dmz, curr_wtp)] * (1- distribution_loss.loc[curr_dmz, curr_wtp]) * (linkage.loc[curr_dmz, curr_wtp])

0.6945600000000001*Volume_('DMZ002',_'Ampang_Intake') + 0.0

In [73]:
# type(demand.loc["DMZ1", "Demand"])
# type(1-distribution_loss.loc["DMZ1", wtp[0]])

In [74]:
capacity.loc["Ampang Intake", "Capacity"]

26411595.83813

In [75]:
distribution_loss.loc["DMZ001", "Ampang Intake"]

0.31838

In [76]:
# ## Meet demand for each DMZ
# for d in dmz:
#     for w in wtp:
#         # No linkage constraint
#         if linkage.loc[d, w] == 0 :
#             model += output[(d,w)] == 0
    
#     model += LpConstraint(
#         lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w])) for w in wtp]),
#         sense=LpConstraintGE,
#         rhs= demand.loc[d, "Demand"],
#         name=d
#         )


In [77]:
# # Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"]

## Within the WTP capacity
for w in wtp:
    model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"]

## No linkage constraint





# Adding Constraints


# ## Within the WTP capacity
# for w in wtp:
#     model += LpConstraint(
#         lpSum([output[(d, w)] for d in dmz]),
#         sense = LpConstraintLE,
#         rhs = capacity.loc[w, "Capacity"]*1000,
#         name = w.replace(" ", "")
#         )





In [78]:
# # Solve the model
# solver = CPLEX_PY()
# solver.buildSolverModel(model)

# #Modify the solvermodel
# solver.solverModel.parameters.timelimit.set(1200)

# #Solve P
# solver.callSolver(model)
# status = solver.findSolutionValues(model)

In [79]:
# # Solve the model
# model.solve(PULP_CBC_CMD(maxSeconds=1000, msg=1, fracGap=0))

In [80]:
# Solve the model
model.solve()

1

In [81]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 1,678,863,553 ($/Month)

Status: Optimal


In [195]:
soln_dict = {i.name: i.varValue for i in model.variables()}

In [196]:
not_satisfied = []

for c in model.constraints.values():
    c_dict = c.toDict()
    # print(c_dict)

    satisfied = False
    
    LHS = sum([soln_dict[i['name']]*i['value'] for i in c_dict['coefficients']])
    LHS = LHS + c_dict['constant']
    
    if c_dict['sense'] == 0:
        satisfied = (LHS == 0)
   
    if c_dict['sense'] == -1:
        satisfied = (LHS <= 0)
    
    if c_dict['sense'] == 1:
        satisfied = (LHS >= 0)
   
    # print(c)
    
    if satisfied:
        #print('LHS: ', LHS)
        print('is satisfied')
    else:
        #print('LHS: ', LHS)
        print(c_dict["name"],'not satisfied', "LHS:", LHS)
        not_satisfied.append(c_dict["name"])

None not satisfied LHS: -0.0026547000161372125
None not satisfied LHS: -0.0021680000063497573
is satisfied
None not satisfied LHS: -0.003575199982151389
None not satisfied LHS: -0.002482999989297241
is satisfied
None not satisfied LHS: -0.0008851999882608652
None not satisfied LHS: -0.001965200004633516
None not satisfied LHS: -0.0033360000234097242
is satisfied
None not satisfied LHS: -0.001102800015360117
is satisfied
is satisfied
is satisfied
None not satisfied LHS: -0.0021776000212412328
None not satisfied LHS: -0.0014420000370591879
None not satisfied LHS: -0.003125200018985197
is satisfied
is satisfied
None not satisfied LHS: -0.00326250001671724
is satisfied
is satisfied
None not satisfied LHS: -0.0022785999753978103
is satisfied
is satisfied
is satisfied
None not satisfied LHS: -0.0013524999958463013
None not satisfied LHS: -0.0002556999970693141
is satisfied
None not satisfied LHS: -0.0010919999913312495
is satisfied
None not satisfied LHS: -0.0005463999987114221
is satisfied


In [197]:
"DMZ001" in not_satisfied

False

In [198]:
"DMZ107" in not_satisfied

False

In [199]:
len(not_satisfied)

49

## Results

In [82]:
dict_wtp = {}
dict_dmz = {}

In [83]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [84]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    curr_dmz = combi[0]
    curr_wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[curr_dmz, curr_wtp] = volume


In [85]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WTP1,WTP10,WTP11,WTP12,WTP13,WTP14,...,WTP38,WTP39,WTP4,WTP40,WTP5,WTP6,WTP7,WTP8,WTP9,WangsaMaju
DMZ001,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,219953.31
DMZ002,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ003,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,208480.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ004,0.0,0.0,0.0,263659.52,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ005,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ995,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
DMZ996,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,209710.92
DMZ997,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,338351.29,0.00
DMZ998,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00


In [86]:
# df.columns = ["Ampang Intake", "Batang Kali", "Bername River Head", "Sungai Tengi", "Wangsa Maju"]
# df = df[["Ampang Intake", "Batang Kali", "Bername River Head", "Wangsa Maju", "Sungai Tengi", ]]

In [87]:
# Supply of water from each WTP to each DMZ (including distribution loss)
# df

### Renaming the columns for distribution loss to match result df

In [88]:
col_list = df.columns.tolist()
print(col_list[:5])

temp_col_list = distribution_loss.columns.tolist()
dist_col_list = [w.replace(" ", "") for w in temp_col_list]

print(dist_col_list[:5])

['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'SungaiTengi', 'WTP1']
['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'WTP1', 'WTP2']


In [89]:
result_distribution_loss = distribution_loss.iloc[:1100, :].copy()
result_distribution_loss.columns = dist_col_list

In [90]:
result_distribution_loss = result_distribution_loss[col_list]

### Accounting for distribution loss

In [91]:
useful_amount = 1- result_distribution_loss.values

In [92]:
temp = df.values * useful_amount
temp

array([[     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       , 160044.6269553],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       ...,
       [     0.       ,      0.       ,      0.       , ...,
             0.       , 224536.6830698,      0.       ],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ],
       [     0.       ,      0.       ,      0.       , ...,
             0.       ,      0.       ,      0.       ]])

In [93]:
temp.shape

(1100, 45)

In [94]:
# Supply of water from each WTP to each DMZ, after accounting for water loss
final_df = pd.DataFrame(temp, columns = wtp, index=dmz)
final_df.round(3)

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,160044.627
DMZ002,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ003,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,141891.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ004,0.0,0.0,0.0,194644.004,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ005,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ1096,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ1097,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,145256.269
DMZ1098,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,224536.683,0.000
DMZ1099,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000


In [95]:
# wtp

In [96]:
supply = final_df.sum(axis=1)
supply[:5]

DMZ001    160044.626955
DMZ002    150056.745492
DMZ003    141891.569672
DMZ004    194644.004045
DMZ005    126901.552787
dtype: float64

In [97]:
sum(final_df.loc["DMZ001",:])

160044.6269553

In [98]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [102]:
prob = []
print("       ", "Supply", "            ", "Demand")
for curr_dmz in dmz:
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
        print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
        prob.append(curr_dmz)

        Supply              Demand
DMZ102 172582.4679457 >= 219335.13647 ?
DMZ103 160445.5190752 >= 162679.28167 ?
DMZ105 168450.9581226 >= 192934.71108 ?
DMZ107 136591.2520794 >= 203053.81513 ?
DMZ110 149929.2856942 >= 167461.64841 ?
DMZ113 140725.6186812 >= 170376.29337 ?
DMZ114 149467.339819 >= 200656.84275 ?
DMZ115 127167.63577159999 >= 211408.31361 ?
DMZ117 163592.788647 >= 176386.39911 ?
DMZ118 135291.9598528 >= 148959.6222 ?
DMZ119 170655.204015 >= 184917.60021 ?
DMZ120 173602.939472 >= 174459.2079 ?
DMZ121 138163.51112580003 >= 233416.07347 ?
DMZ122 214317.708506 >= 214621.31332 ?
DMZ123 158390.176092 >= 206664.34339 ?
DMZ124 128367.5133736 >= 190825.24332 ?
DMZ125 140871.021453 >= 168214.77578 ?
DMZ126 170264.1995736 >= 220918.23369 ?
DMZ127 150509.2942473 >= 155538.47874 ?
DMZ130 141110.71831119998 >= 201130.21082 ?
DMZ134 130740.1597814 >= 174478.69247 ?
DMZ135 139266.32086399998 >= 174213.98184 ?
DMZ138 166972.0234672 >= 171396.88993 ?
DMZ139 129930.4707315 >= 138187.97246 

In [99]:
prob = []
for curr_dmz in dmz:
    # print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
    # print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
        print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
        prob.append(curr_dmz)
    # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
    #     print(True)
    # else: 
    #     print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
    #     print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])

DMZ102 172582.4679457 >= 219335.13647 ?
DMZ103 160445.5190752 >= 162679.28167 ?
DMZ105 168450.9581226 >= 192934.71108 ?
DMZ107 136591.2520794 >= 203053.81513 ?
DMZ110 149929.2856942 >= 167461.64841 ?
DMZ113 140725.6186812 >= 170376.29337 ?
DMZ114 149467.339819 >= 200656.84275 ?
DMZ115 127167.63577159999 >= 211408.31361 ?
DMZ117 163592.788647 >= 176386.39911 ?
DMZ118 135291.9598528 >= 148959.6222 ?
DMZ119 170655.204015 >= 184917.60021 ?
DMZ120 173602.939472 >= 174459.2079 ?
DMZ121 138163.51112580003 >= 233416.07347 ?
DMZ122 214317.708506 >= 214621.31332 ?
DMZ123 158390.176092 >= 206664.34339 ?
DMZ124 128367.5133736 >= 190825.24332 ?
DMZ125 140871.021453 >= 168214.77578 ?
DMZ126 170264.1995736 >= 220918.23369 ?
DMZ127 150509.2942473 >= 155538.47874 ?
DMZ130 141110.71831119998 >= 201130.21082 ?
DMZ134 130740.1597814 >= 174478.69247 ?
DMZ135 139266.32086399998 >= 174213.98184 ?
DMZ138 166972.0234672 >= 171396.88993 ?
DMZ139 129930.4707315 >= 138187.97246 ?
DMZ142 151667.6701768 >= 204328.0

In [54]:
len(prob)

0

In [55]:
check_capacity = final_df.sum(axis=0)

In [56]:
prob_wtp = []
for curr_wtp in wtp:
    # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    # print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])
    a = check_capacity[curr_wtp]
    b = capacity.loc[curr_wtp, "Capacity"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a >= b:
        print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
        prob_wtp.append(curr_wtp)
    # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
    #     print(True)
    # else: 
    #     # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    #     print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])

In [234]:
len(prob_wtp)

0

In [236]:
df.head()

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WTP1,WTP10,WTP11,WTP12,WTP13,WTP14,...,WTP38,WTP39,WTP4,WTP40,WTP5,WTP6,WTP7,WTP8,WTP9,WangsaMaju
DMZ001,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219953.31
DMZ002,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ003,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,208480.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ004,0.0,0.0,0.0,263659.52,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ005,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [237]:
final_df.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160044.626955
DMZ002,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ003,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,141891.569672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ004,0.0,0.0,0.0,194644.004045,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ005,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [238]:
# Export results

df.to_excel("../dataset2/result_without_loss.xlsx", index=True)
final_df.to_excel("../dataset2/result.xlsx", index=True)

In [132]:
demand.loc["DMZ005", "Demand"]

126901.55527